In [32]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, roc_auc_score

from torch.utils.data import DataLoader, TensorDataset


In [33]:
# =========================================
# Simulate Realistic Patient Vitals
# =========================================

np.random.seed(42)

num_patients = 4000
timesteps = 24
num_features = 5  # HR, SBP, SpO2, RR, Glucose

X = np.zeros((num_patients, timesteps, num_features))
y = np.zeros(num_patients)

for i in range(num_patients):
    
    # Baseline vitals
    hr_base = np.random.normal(75, 8)
    sbp_base = np.random.normal(120, 10)
    spo2_base = np.random.normal(98, 1)
    rr_base = np.random.normal(16, 2)
    glucose_base = np.random.normal(100, 15)
    
    deteriorating = np.random.rand() < 0.25  # 25% patients deteriorate
    
    for t in range(timesteps):
        hr = hr_base + np.random.normal(0, 2)
        sbp = sbp_base + np.random.normal(0, 3)
        spo2 = spo2_base + np.random.normal(0, 0.5)
        rr = rr_base + np.random.normal(0, 1)
        glucose = glucose_base + np.random.normal(0, 5)
   
        
        # Gradual deterioration pattern
        if deteriorating:
            drift_strength = np.random.uniform(0.3, 0.8)
            hr += t * drift_strength + np.random.normal(0, 2)
            sbp -= t * drift_strength * 0.6 + np.random.normal(0, 3)
            spo2 -= t * drift_strength * 0.2 + np.random.normal(0, 0.8)
            rr += t * drift_strength * 0.4 + np.random.normal(0, 1.5)
            glucose += t * drift_strength * 0.8 + np.random.normal(0, 10)
        if not deteriorating and np.random.rand() < 0.1:
            hr += np.random.normal(10, 5)


        
        X[i, t] = [hr, sbp, spo2, rr, glucose]
    
    if deteriorating:
        y[i] = 1

print("Dataset shape:", X.shape)
print("Deterioration rate:", y.mean())


Dataset shape: (4000, 24, 5)
Deterioration rate: 0.2495


In [34]:
# =========================================
# Train-Test Split
# =========================================

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    stratify=y,
    random_state=42
)

# Convert to tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)

X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64)

print("Train shape:", X_train_tensor.shape)
print("Test shape:", X_test_tensor.shape)


Train shape: torch.Size([3200, 24, 5])
Test shape: torch.Size([800, 24, 5])


In [35]:
# =========================================
# LSTM Classifier
# =========================================

class DeteriorationLSTM(nn.Module):
    def __init__(self, input_size=5, hidden_size=64):
        super(DeteriorationLSTM, self).__init__()
        
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=1,
            batch_first=True
        )
        
        self.fc = nn.Sequential(
            nn.Linear(hidden_size, 32),
            nn.ReLU(),
            nn.Linear(32, 1),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        out, _ = self.lstm(x)
        out = out[:, -1, :]
        out = self.fc(out)
        return out.squeeze()

model = DeteriorationLSTM()

criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

print("LSTM classifier initialized.")


LSTM classifier initialized.


In [36]:
# =========================================
# Train LSTM Classifier
# =========================================

epochs = 10

for epoch in range(epochs):
    model.train()
    running_loss = 0
    
    for sequences, labels in train_loader:
        optimizer.zero_grad()
        
        outputs = model(sequences)
        loss = criterion(outputs, labels)
        
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    print(f"Epoch [{epoch+1}/{epochs}] Loss: {running_loss/len(train_loader):.4f}")


Epoch [1/10] Loss: 0.5770
Epoch [2/10] Loss: 0.3530
Epoch [3/10] Loss: 0.2244
Epoch [4/10] Loss: 0.1573
Epoch [5/10] Loss: 0.1218
Epoch [6/10] Loss: 0.0918
Epoch [7/10] Loss: 0.0801
Epoch [8/10] Loss: 0.0986
Epoch [9/10] Loss: 0.0767
Epoch [10/10] Loss: 0.0498


In [37]:
# =========================================
# Evaluate LSTM Classifier
# =========================================

model.eval()

all_preds = []
all_probs = []
all_labels = []

with torch.no_grad():
    for sequences, labels in test_loader:
        outputs = model(sequences)
        probs = outputs.numpy()
        threshold = 0.1
        preds = (probs > threshold).astype(int)
        
        all_probs.extend(probs)
        all_preds.extend(preds)
        all_labels.extend(labels.numpy())

accuracy = accuracy_score(all_labels, all_preds)
precision = precision_score(all_labels, all_preds)
recall = recall_score(all_labels, all_preds)
auc = roc_auc_score(all_labels, all_probs)

print("Test Accuracy:", round(accuracy, 4))
print("Precision:", round(precision, 4))
print("Recall:", round(recall, 4))
print("ROC-AUC:", round(auc, 4))


Test Accuracy: 0.9425
Precision: 0.8182
Recall: 0.99
ROC-AUC: 0.9974
